# Image processing and Features extraction

In [1]:
from torchvision import datasets
import PIL
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
from skimage.filters import gabor
import numpy as np
import pickle
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display
import random
%matplotlib inline

C:\Users\Thelee\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
def beep_beep():
    import winsound
    frequency = 2500  # Set Frequency To 2500 Hertz
    duration = 100  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)
    winsound.Beep(frequency, duration)
    winsound.Beep(frequency, duration)

In [ ]:
# import glob
# def make_flist(path):
#     files = [f for f in glob.glob(path + "**/*.bmp", recursive=True)]
#     df=pd.DataFrame(files)
#     df.columns=['img_name']
#     df.to_csv(path+'/'+path[2:]+'.csv',index=False)
# make_flist(path = './C1')
# make_flist(path = './C2')
# make_flist(path = './C3')
# make_flist(path = './C4')
# make_flist(path = './C5')
# make_flist(path = './C6')
# make_flist(path = './C7')
# make_flist(path = './C8')


## Function for image processing and extracting features from all images in the dataset

In [3]:
def feat_all(csv_path,incat=1):
    dfin=pd.read_csv(csv_path)
    # Feature extraction from multiple images
    cat=[]
    lbpene=[]
    lbpent=[]
    contra=[]
    dissim=[]
    homogen=[]
    ener=[]
    corre=[]
    gabor_ene=[]
    gabor_ent=[]
    f = IntProgress(min=0, max=len(dfin)) # instantiate the bar
    display(f) # display the bar
    for i in dfin.img_name:
        img = cv2.imread(i,0)
        #'Finding LBP'
        feat_lbp = local_binary_pattern(img,8,1,'uniform') #Radius = 1, No. of neighbours = 8
        feat_lbp = np.uint8((feat_lbp/feat_lbp.max())*255) #Converting to unit8
        lbp_img = PIL.Image.fromarray(feat_lbp) #Conversion from array to PIL image
        #'Energy and Entropy of LBP feature'
        lbp_hist,_ = np.histogram(feat_lbp,8)
        lbp_hist = np.array(lbp_hist,dtype=float)
        lbp_prob = np.divide(lbp_hist,np.sum(lbp_hist))
        lbp_energy = np.sum(lbp_prob**2)
        lbp_entropy = -np.sum(np.multiply(lbp_prob,np.log2(lbp_prob)))
        #print('LBP energy = '+str(lbp_energy))
        #print('LBP entropy = '+str(lbp_entropy))

        #'Finding GLCM features from co-occurance matrix'
        gCoMat = greycomatrix(img, [2], [0],256,symmetric=True, normed=True) # Co-occurance matrix
        contrast = greycoprops(gCoMat, prop='contrast')
        dissimilarity = greycoprops(gCoMat, prop='dissimilarity')
        homogeneity = greycoprops(gCoMat, prop='homogeneity')
        energy = greycoprops(gCoMat, prop='energy')
        correlation = greycoprops(gCoMat, prop='correlation')
        
        # Gabor filter
        gaborFilt_real,gaborFilt_imag = gabor(img,frequency=0.6)
        gaborFilt = (gaborFilt_real**2+gaborFilt_imag**2)//2


        #'Energy and Entropy of Gabor filter response'
        gabor_hist,_ = np.histogram(gaborFilt,8)
        gabor_hist = np.array(gabor_hist,dtype=float)
        gabor_prob = np.divide(gabor_hist,np.sum(gabor_hist))
        gabor_energy = np.sum(gabor_prob**2)
        gabor_entropy = -np.sum(np.multiply(gabor_prob,np.log2(gabor_prob)))

        lbpene.append(lbp_energy)
        lbpent.append(lbp_entropy)

        contra.append(contrast[0][0])
        dissim.append(dissimilarity[0][0])
        homogen.append(homogeneity[0][0])
        ener.append(energy[0][0])
        corre.append(correlation[0][0])

        gabor_ene.append(gabor_energy)
        gabor_ent.append(gabor_entropy)
        cat.append(incat)
        df = pd.DataFrame({'lbpene': lbpene,
                   'lbpent' : lbpent,
                   'dissim' : dissim,
                   'homogen' : homogen,
                  'ener':ener,
                  'corre':corre,
                  'gabor_ene':gabor_ene,
                  'gabor_ent':gabor_ent,
                  'cat':cat}, columns=['lbpene','lbpent','dissim','homogen','ener','corre','gabor_ene','gabor_ent','cat'])
        df.to_csv('./data/test'+csv_path[5:],index=False)
        f.description=random.choice(['-','/','|','\\'])
        f.value += 1
    print('Filename CSV saved:',csv_path[5:])
    f.description='Done:'

## Save extracted features from all images to csv

In [ ]:
csvpath='./C1/C1.csv'
feat_all(csv_path=csvpath,incat=1)
csvpath='./C2/C2.csv'
feat_all(csv_path=csvpath,incat=2)
csvpath='./C3/C3.csv'
feat_all(csv_path=csvpath,incat=3)
csvpath='./C4/C4.csv'
feat_all(csv_path=csvpath,incat=4)
csvpath='./C5/C5.csv'
feat_all(csv_path=csvpath,incat=5)
csvpath='./C6/C6.csv'
feat_all(csv_path=csvpath,incat=6)
csvpath='./C7/C7.csv'
feat_all(csv_path=csvpath,incat=7)
csvpath='./C8/C8.csv'
feat_all(csv_path=csvpath,incat=8)
beep_beep()